In [17]:
import secrets
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

BLOCK_SIZE = 16  # Assuming a block size of 16 bytes
allzero = bytes([0]*BLOCK_SIZE)
msg1 = b"YourMessage1"
msg2 = b"YourMessage2"
KEY_SIZE = 128  # Assuming a 128-bit key size
key = secrets.token_bytes(KEY_SIZE // 8)

def xorblock(b1, b2: bytes) -> bytes:
    assert len(b1) == len(b2), "xorblock: b1 and b2 must have the same length"
    bxor = bytearray(len(b1))
    for i in range(len(b1)):
        bxor[i] = b1[i] ^ b2[i]
    return bytes(bxor)

def E(k, m: bytes) -> bytes:
    """AES-128 in ECB mode"""
    assert len(m) == 16, "E(k,m): Input block m must be 16 bytes long."
    assert len(k) == 16, "E(k,m): key must be 16 bytes long."
    encryptor = Cipher(algorithms.AES(k), modes.ECB()).encryptor()
    return (encryptor.update(m) + encryptor.finalize())

def CBC_MAC_block(key, tag, msg: bytes) -> bytes:
    """1-block CBC-MAC"""
    if len(msg) >= BLOCK_SIZE:
        cmb = msg[:BLOCK_SIZE]
        msg = msg[BLOCK_SIZE:]
    else:
        cmb = pad(msg)
        msg = b''
    return E(key, xorblock(tag, cmb)), msg

def CBC_MAC(key, msg: bytes) -> bytes:
    tag = allzero
    while len(msg) > 0:
        tag, msg = CBC_MAC_block(key, tag, msg)
    return tag

def b2a(b: bytes) -> str:
    """Bytes to ASCII. For 16-byte objects only."""
    assert len(b) == BLOCK_SIZE, "Length must be 16 bytes -- was {:d} for {:s}".format(len(b), str(b))
    hexstr = b.hex()
    hexstr = hexstr[0:8] + " " + hexstr[8:16] + " " + hexstr[16:24] + " " + hexstr[24:]
    return hexstr

if __name__ == "__main__":
    print("\n*** Is (t2 == m1 || (m2 ^ t1)) ? \n")
    msg1 = pad(msg1)
    msg2 = pad(msg2)
    print("Key : ", b2a(key))
    print("msg1 : ", len(msg1), b2a(msg1))
    print("msg2 : ", len(msg2), b2a(msg2))
    tag1 = CBC_MAC(key, msg1)
    tag2 = CBC_MAC(key, msg2)
    print("tag1 : ", len(tag1), b2a(tag1))
    print("tag2 : ", len(tag2), b2a(tag2))
    print("\nNow we construct msg3 to be: m1 || (m2 ^ t1)\n")
    msg3 = msg1 + xorblock(msg2, tag1)
    print("msg3 : ", len(msg3), b2a(msg3[:BLOCK_SIZE]), b2a(msg3[BLOCK_SIZE:]))
    tag3 = CBC_MAC(key, msg3)
    print("tag3 : ", len(tag3), b2a(tag3))
    print("\nThen we have: (tag2 == tag3) is", tag3 == tag2)



*** Is (t2 == m1 || (m2 ^ t1)) ? 

Key :  aa321ec6 9fc4284a 5b8d2f5b 51feabc7
msg1 :  16 596f7572 4d657373 61676531 ffffffff
msg2 :  16 596f7572 4d657373 61676532 ffffffff
tag1 :  16 00e58b6b 4d5f2b1d 3e875664 7d1711dd
tag2 :  16 4efe404f cb3fa4b7 12822e34 f9dfa34f

Now we construct msg3 to be: m1 || (m2 ^ t1)

msg3 :  32 596f7572 4d657373 61676531 ffffffff 598afe19 003a586e 5fe03356 82e8ee22
tag3 :  16 4efe404f cb3fa4b7 12822e34 f9dfa34f

Then we have: (tag2 == tag3) is True
